In [1]:
%%writefile T2mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

StopWordsFile = 'stop_words_en.txt'
StopWordsSet = set()

# Makes a list containing the stop words
with open(StopWordsFile, 'r') as f_stop:
    for line in f_stop:
        StopWordsSet.add(line.strip())
f_stop.close()

total_words, stop_words = 0,0
for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        if word.lower() in StopWordsSet:
            stop_word_test = 1
        else:
            stop_word_test = 0
        print >> sys.stderr, "reporter:counter:Stop stats,Stop words,%d" % stop_word_test

Writing T2mapper.py


In [2]:
%%writefile WordPerc.py

import sys
index = 0
WordTypeList = ['Stop', 'Total']
WordDict = {}
for line in sys.stdin:
    WordDict[WordTypeList[index]] = int((line.strip()).split('=')[1])
    index += 1
print ("%.4f" % (((100.0 * WordDict['Stop']) / WordDict['Total'])))

Writing WordPerc.py


In [4]:
%%writefile Calc.sh
cat output.log | grep words= | python WordPerc.py
cat output.log >&2

Writing Calc.sh


In [5]:
%%bash
chmod 777 Calc.sh

In [6]:
%%bash

OUT_DIR="wordstop_result_"$(date +"%s%6N")
NUM_REDUCERS=0

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming Stopwords" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files T2mapper.py,/datasets/stop_words_en.txt \
    -mapper "python T2mapper.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> output.log
./Calc.sh

41.6026


18/02/07 03:31:56 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/02/07 03:31:57 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/02/07 03:31:57 INFO mapred.FileInputFormat: Total input files to process : 1
18/02/07 03:31:57 INFO mapreduce.JobSubmitter: number of splits:2
18/02/07 03:31:57 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1517949445965_0002
18/02/07 03:31:57 INFO impl.YarnClientImpl: Submitted application application_1517949445965_0002
18/02/07 03:31:57 INFO mapreduce.Job: The url to track the job: http://7588fa1c8798:8088/proxy/application_1517949445965_0002/
18/02/07 03:31:57 INFO mapreduce.Job: Running job: job_1517949445965_0002
18/02/07 03:32:02 INFO mapreduce.Job: Job job_1517949445965_0002 running in uber mode : false
18/02/07 03:32:02 INFO mapreduce.Job:  map 0% reduce 0%
18/02/07 03:32:19 INFO mapreduce.Job:  map 43% reduce 0%
18/02/07 03:32:25 INFO mapreduce.Job:  map 65% reduce 0%
18/02/07 03:32:31 INFO 